<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/Predicci%C3%B3n_y_salida_de_Reconocimiento_Acordes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from google.colab import drive
import itertools
import math
from IPython.display import Audio
import math

import tempfile
import os
import keras.models

import cv2
from PIL import Image


Librosa(https://zenodo.org/badge/6309729.svg)


Carga del fichero de audio a predecir

In [9]:
drive.mount('/content/drive')
audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Dover - King George Acústico 2012.mp3'
# audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/02 - Eric Clapton - Before You Accuse Me .mp3'
#audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Lynyrd Skynyrd - 01 - Sweet Home Alabama.mp3'
# Carga en la librería Librosa
audio_data, sample_rate = librosa.load(audio_file)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carga del modelo preentrenado

In [11]:
model = keras.models.load_model('/content/drive/MyDrive/proyecto deteccion acordes/modelo_entrenado/recon_acordes.h5')


Array de acordes soportados, en el mismo orden en que se entrenó el modelo

In [30]:
lista_acordes= ['Am', 'Bb', 'C', 'Dm', 'Bdim', 'G', 'F', 'Em']
acordes =np.array(lista_acordes)

**Tratamiento del fichero de audio**:

1.   Extraer los datos del fichero de audio, de la misma manera que cuando se entrenó el modelo

    -   División del fichero en muestras de medio segundo
    -   Normalización de la muestra a imagen en blanco y negro
    -   Reescalado de la imagen a 300 x 300 píxeles

2.   Predecir cada muestra de tiempo de la canción
3.   Generar un fichero de salida con el tiempo de inicio y fin de la muestra y el acorde predicho


In [34]:
# Divide el audio en segmentos de 1/2 segundo y genera histogramas y periodogramas
segment_duration = 0.5  # Duración del segmento en segundos
num_segments = int(len(audio_data) / (sample_rate * segment_duration))
salida = []
for i in range(num_segments):
    start_time = i * segment_duration
    end_time = (i + 1) * segment_duration
    segment = audio_data[int(start_time * sample_rate):int(end_time * sample_rate)]

    # Deshabilitar la visualización interactiva de las imágenes
    plt.ioff()
    #separar la parte armónica de la percusiva del audio de origen
    audio_harmonic, audio_percussive = librosa.effects.hpss(audio_data)

    # Periodograma del segmento
    plt.figure(figsize=(4, 4))
    librosa.display.specshow(librosa.power_to_db(abs(librosa.stft(audio_harmonic)), ref=np.max), y_axis='log', x_axis='time', cmap='viridis')
    plt.axis('off')
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
     # Guardar la imagen
    nombreimagen = os.path.join('/content/', 'imagen.png')
    plt.savefig(nombreimagen, bbox_inches='tight', pad_inches=0)
    normalizar_imagen('/content/')
    #cargar imagen en X
    X = load_dataset('/content/')

    #hacer predict con el modelo
    pred = model.predict(X)
    #añadir linea con el predict a un fichero de texto
    row = []
    row.append(start_time)
    row.append(end_time)
    predicho = np.argmax(pred)
    print(pred)
    print(predicho)

    row.append(acordes[predicho])
    salida.append (row)
    print(row)

# Escribir los datos en el archivo de texto
with open('/content/salida.srt', 'w') as file:
    for row in salida:
        row_str = ' '.join(map(str, row))  # Convertir la fila a una cadena de texto separada por espacios
        file.write(row_str + '\n')  # Escribir la fila en el archivo



1/1 [==============================] - 0s 62ms/step
[[9.8499632e-01 9.5103751e-06 3.1895652e-06 1.6560683e-03 1.2768309e-02
  5.3074094e-04 5.3654308e-06 3.0489849e-05]]
0
[0.0, 0.5, 'Am']
1/1 [==============================] - 0s 64ms/step
[[9.8499632e-01 9.5103751e-06 3.1895652e-06 1.6560683e-03 1.2768309e-02
  5.3074094e-04 5.3654308e-06 3.0489849e-05]]
0
[0.5, 1.0, 'Am']
1/1 [==============================] - 0s 64ms/step
[[9.8499632e-01 9.5103751e-06 3.1895652e-06 1.6560683e-03 1.2768309e-02
  5.3074094e-04 5.3654308e-06 3.0489849e-05]]
0
[1.0, 1.5, 'Am']
1/1 [==============================] - 0s 64ms/step
[[9.8499632e-01 9.5103751e-06 3.1895652e-06 1.6560683e-03 1.2768309e-02
  5.3074094e-04 5.3654308e-06 3.0489849e-05]]
0
[1.5, 2.0, 'Am']


KeyboardInterrupt: 

In [13]:
nombreimagen

'/content/imagen.png'

Cargar imagen en un dataset, para predecir

In [33]:
def load_dataset(dirname):
  X = []
  entorno =os.path.basename(os.path.dirname(dirname))
  files = os.listdir(os.path.join(dirname))
  images = [file for file in files if file.lower().endswith("png")]
  i=0
  for image_name in images:
    i+=1
    imagen = cv2.imread(os.path.join(dirname, image_name))
    imagen = cv2.resize(imagen, (300,300), interpolation=cv2.INTER_CUBIC)

  X.append(imagen)
  return np.array(X)

Paso la imagen a escala de grises, antes de evaluarla

In [21]:
def normalizar_imagen(dirname):
  files = os.listdir(os.path.join(dirname))
  images = [file for file in files if file.lower().endswith("png")]
  for image_name in images:
    imagen = cv2.imread(os.path.join(dirname, image_name))
    # Convierto todas las imágenes a escala de grises
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY) # Normalio todas las imágenes a escala de grises
    imagen_gris = Image.fromarray(imagen_gris)
    imagen_gris.save(os.path.join( dirname, image_name))